In [1]:
#Install Dependencies
!pip install pyyaml


In [12]:
import client.domino_utils as domino_utils

In [13]:
#Domino users list
user_list = domino_utils.get_user_names_list()
print(user_list)

['integration-test', 'wadkars', 'hw-tier-1', 'test-user-1', 'test-org', 'elaine', 'ahmet']


In [ ]:
#Domino projects list
projects_list = domino_utils.get_all_project_names()
print(projects_list)

Manage you HW Tiers using the JSON below

First note the `rules` section-

- `description` - This is the error message that is applied to the workspace logs when the workspace launch fails if a not permitted user attempts to launch a workspace in using the hwtier described in the `hwtier_id`.

- `projects` - The projects inside which the `hwtier_id` can be used to launch a workspace. If empty, it implies that the use of the `hwtier_id` is permitted in all projects for the permitted `users`.

- `users` - The users who are permitted to launch a worksapce  in the permitted `projects`. If empty, all users are permitted to launch a workspace using the `hwtier_id` in the permitted projects.

Next let us review the outer section of this JSON

- `replacement_hw_tier` - Use the smallest HWTier here. The intent is that this mutation fails the workspace after it launches for large HWTiers. This attribute changes the POD definition to use the smallest possible HWTier to launch the workspace to prevent cost implications starting a workspace on a large hwtier only to fail the workspace. The intent of this mutation is to only use these HWTiers in exceptional circumstances. Hence it is possible that new nodes will spin to meet the capacity requirements of this HWTier. This results in unncessary costs.
- `busybox_image` - The busybox image path to use
- `command` - The path to the shell. The default values are good enough
- `message_prefix` - The error message in the `description` is prefixed with this string. The intent is to help support to identify the root cause of why thie workspace launch failed so they can tell the customer that the mutation caused it.

In [5]:
##Update your policies here
hwtier_policies={
  "replacement_hw_tier": "small-k8s",
  "busybox_image": "busybox",
  "command" : [ "/bin/sh","-c" ],
  "message_prefix" : "HWTIER-ACCESS-POLICY",
  "rules":[
    {
      "description": "HWTier cannot be used by this user for this project",
      "hwtier_id": "extra-large-k8s",
      "projects": ["basavpr/quick-start","test-user-2/quickstart"],
      "users": ["basavpr2","test-user-1","test-user-3"]
    }
  ]
}


In [2]:
import json
import sys
import client.domsed_client as dc
import client.domino_utils as domino_utils


In [3]:
##Templates- DO NOT MODIFY
mutation = {
  "apiVersion": "apps.dominodatalab.com/v1alpha1",
  "kind": "Mutation",
  "metadata": {
    "name": "restrict-hw-tier"
  },
  "rules": []
}


In [4]:
mutation_rules_template={
   "containerImageRewriteRule" : {
         "containerImageRewrite":{
            "args":[
               "echo HWTier-Restrict-Mutation- Use of this hardware tier is not permitted &2; exit 1;"
            ],
            "command":[
               "/bin/sh",
               "-c"
            ],
            "name":"run",
            "newImage":"",
            "originalImage":""
         },
         "hardwareTierIdSelector":[
         ],         
         "matchBuilds":False,
      },
      "modifyHWTierRule" : {
         "modifyHWTier":{
            "hwtier_id":""
         },
         "hardwareTierIdSelector":[
         ],
         "matchBuilds":False,
      }
}

In [6]:
replacement_tier = hwtier_policies['replacement_hw_tier']
busybox_image = hwtier_policies['busybox_image']
command = hwtier_policies['command']
message_prefix = hwtier_policies['message_prefix']

In [7]:
def resetSelectors(rule):
    rule['notProjectSelector'] = []
    rule['notUsernameSelector'] = []
    rule['projectSelector'] = []
    rule['usernameSelector'] = []
    return rule

In [8]:
mutation_rules=[]

for r in hwtier_policies["rules"]:
    container_image_rewrite_rule = mutation_rules_template['containerImageRewriteRule']
    description = r['description']
    container_image_rewrite_rule['containerImageRewrite']['command'] =hwtier_policies['command']
    container_image_rewrite_rule['containerImageRewrite']['args']= [f"echo {message_prefix}-{description} >&2; exit 1;"]
    container_image_rewrite_rule['containerImageRewrite']['newImage'] = busybox_image
    container_image_rewrite_rule['hardwareTierIdSelector']=[r['hwtier_id']]
    
    
    rule = container_image_rewrite_rule.copy()
    rule['notProjectSelector'] = r['projects']
    rule['notUsernameSelector'] = r['users']    
    mutation_rules.append(rule)
    
    rule = container_image_rewrite_rule.copy()
    rule['projectSelector'] = r['projects']
    rule['notUsernameSelector'] = r['users']    
    mutation_rules.append(rule)
    
    rule = container_image_rewrite_rule.copy()
    rule['notProjectSelector'] = r['projects']
    rule['usernameSelector'] = r['users']    
    mutation_rules.append(rule)
    
    if('users' in r and len(r['users'])>0):        
        modifyhwtier_rule = mutation_rules_template['modifyHWTierRule']
        modifyhwtier_rule['modifyHWTier']['hwtier_id']=replacement_tier
        modifyhwtier_rule['hardwareTierIdSelector']=[r['hwtier_id']]
        
        rule = modifyhwtier_rule.copy()
        
        rule['notProjectSelector'] = r['projects']
        rule['notUsernameSelector'] = r['users']        
        mutation_rules.append(rule)
        
        rule = modifyhwtier_rule.copy()
        rule['projectSelector'] = r['projects']
        rule['notUsernameSelector'] = r['users']    
        mutation_rules.append(rule)
        
        rule = modifyhwtier_rule.copy()
        rule['notProjectSelector'] = r['projects']
        rule['usernameSelector'] = r['users']    
        mutation_rules.append(rule)



In [9]:
mutation["rules"] = mutation_rules
json_formatted_str = json.dumps(mutation, indent=2)
print(json_formatted_str)

{
  "apiVersion": "apps.dominodatalab.com/v1alpha1",
  "kind": "Mutation",
  "metadata": {
    "name": "restrict-hw-tier"
  },
  "rules": [
    {
      "containerImageRewrite": {
        "args": [
          "echo HWTIER-ACCESS-POLICY-HWTier cannot be used by this user for this project >&2; exit 1;"
        ],
        "command": [
          "/bin/sh",
          "-c"
        ],
        "name": "run",
        "newImage": "busybox",
        "originalImage": ""
      },
      "hardwareTierIdSelector": [
        "extra-large-k8s"
      ],
      "matchBuilds": false,
      "notProjectSelector": [
        "basavpr/quick-start",
        "test-user-2/quickstart"
      ],
      "notUsernameSelector": [
        "basavpr2",
        "test-user-1",
        "test-user-3"
      ]
    },
    {
      "containerImageRewrite": {
        "args": [
          "echo HWTIER-ACCESS-POLICY-HWTier cannot be used by this user for this project >&2; exit 1;"
        ],
        "command": [
          "/bin/sh",
      

In [11]:
dc.apply(mutation)

2022-08-05 20:24:24,237 - domsed_client - WARNING - Publishing Mutation To Domsed


Applied Mutation
{"apiVersion":"apps.dominodatalab.com/v1alpha1","kind":"Mutation","metadata":{"creationTimestamp":"2022-08-05T20:24:24Z","generation":1,"managedFields":[{"apiVersion":"apps.dominodatalab.com/v1alpha1","fieldsType":"FieldsV1","fieldsV1":{"f:rules":{}},"manager":"OpenAPI-Generator","operation":"Update","time":"2022-08-05T20:24:24Z"}],"name":"restrict-hw-tier","namespace":"domino-platform","resourceVersion":"19193978","uid":"2e03bafb-0e5a-43c5-be91-996995fbf3b7"},"rules":[{"containerImageRewrite":{"args":["echo HWTIER-ACCESS-POLICY-HWTier cannot be used by this user for this project >&2; exit 1;"],"command":["/bin/sh","-c"],"name":"run","newImage":"busybox","originalImage":""},"hardwareTierIdSelector":["extra-large-k8s"],"matchBuilds":false,"notProjectSelector":["basavpr/quick-start","test-user-2/quickstart"],"notUsernameSelector":["basavpr2","test-user-1","test-user-3"]},{"containerImageRewrite":{"args":["echo HWTIER-ACCESS-POLICY-HWTier cannot be used by this user for t